In [638]:
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go
import numpy as np
from math import e
ln = np.log  
ln(e)

1.0

In [639]:
df = pd.read_excel("extrem.xlsx")

In [640]:
df.head()

,Datetime,Unnamed: 1,temp,Unnamed: 3,tnz,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,tt,t_tenzo,tnzl,dif
0,2024-03-18 14:06:40,8,34.6248,26.800,2183140,2178584,2178557,0,0,42.61,36.4642,26,52.15,0,-163,35.108667,2183140,0
1,2024-03-18 14:06:52,8,35.1197,24.120,2183170,2178549,2178525,0,0,38.31,36.9437,24,52.47,0,-861,35.574000,2183170,30
2,2024-03-18 14:08:05,8,35.5940,21.708,2183159,2178476,2178503,0,0,37.17,37.4262,-27,52.73,0,-1603,36.068667,2183159,-11
3,2024-03-18 14:09:17,8,36.0510,19.537,2183152,2178408,2178484,0,0,37.02,37.8699,-75,52.96,0,-2266,36.510667,2183152,-7
4,2024-03-18 14:10:30,8,36.4945,17.584,2183170,2178368,2178466,0,0,37.15,38.3060,-98,53.17,0,-2935,36.956667,2183170,18


In [641]:
desired_columns = ['Datetime','t_tenzo','tnzl']

In [642]:
df = df.drop(df.columns.difference(desired_columns),axis=1)

In [643]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  909 non-null    datetime64[ns]
 1   t_tenzo   909 non-null    float64       
 2   tnzl      909 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 21.4 KB


In [644]:
df.head()

,Datetime,t_tenzo,tnzl
0,2024-03-18 14:06:40,35.108667,2183140
1,2024-03-18 14:06:52,35.574000,2183170
2,2024-03-18 14:08:05,36.068667,2183159
3,2024-03-18 14:09:17,36.510667,2183152
4,2024-03-18 14:10:30,36.956667,2183170


In [645]:
def create_plot(df):
    df.loc[150:, 'tnzl'] += 80

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df['Datetime'], y=df['tnzl'], mode='lines', name='tenzo'))
    fig.add_trace(go.Scatter(x=df['Datetime'], y=df['t_tenzo'], mode='lines', name='Temp', yaxis='y2'))

    fig.update_layout(
        title='График Temp и tenzo от времени',
        title_x=0.5,
        xaxis=dict(title='Datetime'),
        yaxis=dict(tickformat='d', title='Tenzo', side='left', visible=True),
        yaxis2=dict(tickformat='d', title='Temp', overlaying='y', side='right', visible=True),
        hovermode="x",
        font=dict(size=14),
        margin=dict(l=100, r=100, t=50, b=50)
    )

    fig.show()


In [646]:
create_plot(df)

In [647]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  909 non-null    datetime64[ns]
 1   t_tenzo   909 non-null    float64       
 2   tnzl      909 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 21.4 KB


In [648]:
pearson_corr = df['t_tenzo'].corr(df['tnzl'], method='pearson')
spearman_corr = df['t_tenzo'].corr(df['tnzl'], method='spearman')

print("Коэффициент Пирсона:", pearson_corr)
print("Коэффициент Спирмена:", spearman_corr)

Коэффициент Пирсона: 0.781616267689445
Коэффициент Спирмена: 0.8426040093870645


In [649]:
def objective_function(C, df):

    t_exp = df.t_tenzo * (1 - np.exp(-abs(C[0])))  

    dt = df['t_tenzo'].diff() * C[1]

    U_ln = df['tnzl'] - C[2] * ln(t_exp)  

    dU = U_ln.diff() * C[3]
    
    dU_meam = dU.rolling(window=7).mean()

    dU_meam_dt = dU_meam + C[4]*dt

    dU_meam_dt = U_ln + dU_meam_dt * C[5]

    
    valid_seconds = list(range(1, 910))
    U_ln_dU_meam_dt = dU_meam_dt - C[6] * ln(valid_seconds)

    var_level = np.var(U_ln_dU_meam_dt)
    
    return var_level

initial_guess = [0.03, 2.43, 2791, 0.000027, 0.000096, 446459.39, 324.27]

result = minimize(objective_function, initial_guess, args=(df,), method='BFGS')

optimal_K = result.x
optimal_scalar_value = objective_function(optimal_K, df)

print('Оптимальные значения K:', pd.DataFrame(optimal_K))
print('Оптимальные значения дисперсии:', optimal_scalar_value)


Оптимальные значения K:                0
0      -0.271431
1       3.176577
2    2787.487201
3       0.000023
4       0.000002
5  446459.424491
6     326.746143
Оптимальные значения дисперсии: 932.991949601556


In [650]:
def function(C, df):

    t_exp = df.t_tenzo * (1 - np.exp(-abs(C[0])))  

    dt = df['t_tenzo'].diff() * C[1]

    U_ln = df['tnzl'] - C[2] * ln(t_exp)  

    dU = U_ln.diff() * C[3]
    
    dU_meam = dU.rolling(window=7).mean() 

    dU_meam_dt = dU_meam + C[4]*dt

    dU_meam_dt = U_ln + dU_meam_dt * C[5]

    valid_seconds = list(range(1, 910))
    U_ln_dU_meam_dt = dU_meam_dt - C[6] * ln(valid_seconds)


    return U_ln_dU_meam_dt

level_dHdt = function(result.x,df)

optimal_K_list = [optimal_K]
optimal_K_df = pd.DataFrame(optimal_K_list,columns=['K{}'.format(i) for i in range(0, 7)])
optimal_K_df.to_csv('K0.csv', mode='a', header=False, index=False)


In [651]:
level_dHdt.info()

<class 'pandas.core.series.Series'>
RangeIndex: 909 entries, 0 to 908
Series name: None
Non-Null Count  Dtype  
--------------  -----  
902 non-null    float64
dtypes: float64(1)
memory usage: 7.2 KB


In [652]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Datetime'], y=df.tnzl, mode='lines', name='Tenzo Data'))
fig.add_trace(go.Scatter(x=df['Datetime'], y=level_dHdt, mode='lines', name='Level Data', yaxis='y2'))
    
  
fig.update_layout(
    legend=dict(x=.8),
    title='График сравнения  Tenzo и Level',
    title_x=0.5,
    xaxis=dict(title='Datatime'),
    yaxis=dict(tickformat ='d',title='Tenzo Data', side='left'),
    yaxis2=dict(tickformat ='d',title='Level Data', overlaying='y', side='right'),
    hovermode="x",
    font=dict(size=14),
    margin=dict(l=100, r=100, t=50, b=50) 
    )
fig.show()


In [653]:
delta_level = np.nanmax(level_dHdt) - np.nanmin(level_dHdt)
print("Разница уровней =", delta_level)


Разница уровней = 326.1593888080679


In [654]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Datetime'], y=df.tnzl, mode='lines', name='Tenzo Data'))
fig.add_trace(go.Scatter(x=df['Datetime'], y=level_dHdt, mode='lines', name='Level Data'))
    
  
fig.update_layout(
    legend=dict(x=.8),
    title='График сравнения  Tenzo и Level',
    title_x=0.5,
    xaxis=dict(title='Datatime'),
    yaxis=dict(tickformat ='d',title='Tenzo Data', side='left'),
    hovermode="x",
    font=dict(size=14),
    margin=dict(l=100, r=100, t=50, b=50) 
    )
fig.show()
